# Redes neuronales para datos visuales 
En el siguiente ejemplo, utilizaremos redes neuronales convolucionales para categorizar imágenes, utilizando los pixeles de estas como _features_. Es importante, más que en el ejemplo de _embeddings_, que antes de ejecutar el código, estemos utilizando un _Runtime_ de tipo GPU. Para esto, deben seleccionar en el menú de arriba `Runtime -> Change Runtime Type -> GPU` y luego `Save`.

## 1. Importación de librerías
Las librerías que utilizaremos en este ejemplo son similares a las del ejemplo de _embeddings_. La principal diferencia es que acá utilizaremos los modulos de Pytorch especializados en procesar imágenes.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

import numpy as np
import tqdm

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#estas instrucciones sirven para aumentar el tamaño de gráficos e imágenes
plt.rcParams['figure.figsize'] = [15, 10]
plt.rcParams.update({'font.size': 16})

## 2. Lectura y preprocesamiento de datos
A diferencia de los casos anteriores, acá utilizaremos un set que viene precargado en Pytorch, lo que significa que no es necesario crear una clase derivada de `Dataset`.
El set que usaremos es el CIFAR10, que contiene imágenes a color de 32x32 pixeles, pertencecientes a 10 categorías: ‘airplane’, ‘automobile’, ‘bird’, ‘cat’, ‘deer’,‘dog’, ‘frog’, ‘horse’, ‘ship’, ‘truck’.

Un aspecto interesante de Pytorch es que permite definir una secuencia de transformaciones para los datos, de manera de automatizar ciertas partes del preprocesamiento. En este caso, dado que los pixeles  de las imágenes del set de datos viven en el rango [0,1], usaremos la composición definida en la siguiente celda, para crear una transformación que primero lleve las imágenes a tensores de Pytorch, y luego las lleve al rango [-1, 1].



In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

A continuación, definimos los sets de entrenamiento y test, junto con sus respectivos `DataLoaders`. Notemos que el set de datos es descargado directamente por Pytorch, sin requerir mayor intervención nuestra.

In [ ]:
batch_size = 256
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

Finalmente, visualizamos algunas de las imágenes elegidas aleatoriamente. Es importante notar que para poder mostrarlas, es necesario denormalizar las imágenes (función `ishow`, línea 2).



In [ ]:
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

dataiter = iter(trainloader)
images, labels = dataiter.next()
num_images = 8
images = images[:num_images]
labels = labels[:num_images]
imshow(torchvision.utils.make_grid(images))
print('GT:\t', (8*' ').join('%-6s' % classes[labels[j]] for j in range(num_images)))

## 3. Definición del modelo
Para procesar los datos, definimos una red con capas convolucionales densas, cuidando de hacer coincidir de manera adecuada la cantidad de filtros/neuronas de cada una (recuerden las fórmulas vistas en clase). Es importante notar que en el `forward`, luego de las primeras 2 capas convolucionales hay un _max pooling_, lo que afecta en la definición del tamaño de entrada de la primera capa densa.

Un aspecto interesante es que antes de pasar las features a la primera capa densa, es necesario cambiar su forma (función `view`), para transformarlas en un vector unidimensional.

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, 5)
        self.conv2 = nn.Conv2d(16, 32, 3)
        self.conv3 = nn.Conv2d(32, 32, 3)
        self.fc = nn.Linear(32 * 4 * 4, 64)
        self.classifier = nn.Linear(64, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv3(x))
        x = x.view(-1, 32 * 4 * 4)
        x = F.relu(self.fc(x))
        x = self.classifier(x)
        return x

## 4. Instanciación del modelo, pérdida y optimizador
Sin grandes misterios acá, instanciamos el modelo y lo enviamos a la GPU, y luego creamos la pérdida (_Cross entropy_ para clasificación) y el optimizador (Adam).



In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CNN().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

##5. Entrenamiento
El entrenamiento sigue la misma lógica que el ejemplo anterior de Pytorch para datos tabulados, con pequeñas diferencias en la manera en que mostramos el avance del proceso. Con el fin de que no tome tanto el proceso, solo realizaremos 10 épocas de entrenamiento. Este número puede, y debe, modificarse para calibrar de mejor manera el rendimiento, siempre considerando un set de validación.



In [ ]:
num_epochs = 10

for epoch in range(num_epochs):
  total_loss = 0
  total_examples = 0

  with tqdm.notebook.tqdm(total=len(trainloader), unit='batch', desc=f'Epoch {epoch+1}/{num_epochs}', 
                          position=100, leave=True) as pbar:  
    for X, Y in trainloader: 

      X = X.to(device)
      Y = Y.to(device)

      # Forward pass
      Y_ = model(X)
      loss = criterion(Y_, Y)

      # Backward pass
      optimizer.zero_grad()
      loss.backward()

      # Gradient descent
      optimizer.step()

      total_loss += loss.item()*X.size(0)
      total_examples += X.size(0)

      pbar.set_postfix(loss=total_loss/total_examples)
      pbar.update()     

##6. Evaluación
A continuación evaluaremos el rendimiento del modelo recién entrenado. Nuestra primera tarea será visualizar algunas de sus predicciones. Es importante notar que, dado que la red predice una distribución de probabilidades sobre las 10 categorías (_softmax_), debemos reportar la categoría con mayor probabilidad (línea 2).  



In [ ]:
outputs = model(images.to(device))
_, predicted = torch.max(outputs, 1)
imshow(torchvision.utils.make_grid(images))
print('GT:\t', (8*' ').join('%-6s' % classes[labels[j]] for j in range(num_images)))
print('Pred:\t', (8*' ').join('%-6s' % classes[predicted[j]] for j in range(num_images)))

Como se aprecia, la red hace en general un trabajo razonable de predicción, a pesar de que algunas predicciones no tienen mucho sentido. Dado que las imágenes usadas son del mismo set de entrenamiento, es probable que más épocas permitan mejorar esto, potencialmente a costa de un mayor nivel de sobreentrenamiento.

A continuación, evaluaremos el rendimiento en el set de test. Un elemento importante a considerar es que como métrica de rendimiento no utilizaremos la pérdida del modelo, sino algo más "humanamente" informativo como el _accuracy_ (razón entre correcciones correctas y elementos totales de una clase). Un detalle importante del _accuracy_ es que en su versión básica, no toma en consideración si una clase tiene más ejemplos que otra. Esto hace que en problemas con desbalance, esta métrica pueda lleva a engaño (solo la clase mayor está bien clasificada). Si bien en este caso las clases están balanceadas y basta con mantener la versión básica, implementaremos la versión balanceada, calculando primero el _accuracy_ por clase y luego promediándolos. Fuera de eso, el proceso es muy similar al realizado durante el entrenamiento.

**ULTRA IMPORTANTE**: al evaluar, es fundamental cambiar el modelo a modo `eval`. Por un lado, esto permite que el modelo no sea modificable y que se gaste menos memoria, pero por otro lado esto hace que algunas capas (como las de _dropout_ o _batchnorm_), cambien su funcionamiento automaticamente al modo evaluación (no funcionan igual en entrenamiento y evaluación). No confundir las funcionalidades del método `eval()` con el método `torch.no_grad()`, que solo evita que se calcule el gradiente.

In [ ]:
correct = [0]*len(classes)
total = [0]*len(classes)
acc = [0]*len(classes)

model.eval()
with tqdm.notebook.tqdm(total=len(testloader), unit='batch', desc=f'Evaluation', 
                        position=100, leave=True) as pbar:  
  for X, Y in testloader: 

    X = X.to(device)
    Y = Y.to(device)

    _, Y_ = torch.max(model(X).data, 1)

    for y, y_ in zip(Y, Y_):
      if y == y_:
          correct[y] += 1
      total[y] += 1
      acc[y] = correct[y]/total[y]

    pbar.set_postfix(accuracy=sum(acc)/len(classes))
    pbar.update()    

Si bien el accuracy que logramos no se ve muy impresionante, si consideramos que la probabilidad de clasificar correctamente un ejemplo si se hace de manera aleatoria es de 0,1, entonces parece no ser tan malo.

Finalmente, revisaremos el _accuracy_ por clase, con el fin de verificar si hay categorías más complejas que otras:

In [ ]:
for class_name,class_acc in zip(classes,acc):
    print(f'Accuracy para la clase {class_name:5s}: {class_acc:.2f}')
print(f'Accuracy promedio (balanceado): {sum(acc)/len(classes):.2f}')

Podemos ver que hay una disparidad bastante grande en el _accuracy_ de cada clase, detacando lo bajo de _cat_ y lo alto de _car_ y _frog_. Queda como ejercicio propuesto el explorar manera de mejorar el rendimiento en test y el rendimiento en las clases más difíciles.